<a href="https://colab.research.google.com/github/diegoednaldo/LH_CD_DIEGOSANTANA/blob/main/notebook_Modelagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelagem Preditiva IMDB


## Construção do modelo para previsão da nota IMDb
Segue abaixo todo o passo a passo de como fiz construção da minha modelagem preditiva para a previsão da nota IMDb, com todas as resoluções, análises e explicações necessárias.

### Importando as Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
import warnings
warnings.filterwarnings("ignore")
import joblib

### Carregando e Apresentando o Dataset

In [2]:
# Carregar o dataset
df = pd.read_csv('desafio_indicium_imdb.csv', index_col=0)

# Mostrar as primeiras linhas do dataset
df.head()

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
5,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"


### Divisão dos Dados em Conjunto de Treinamento e Teste

Primeiro, separo as características (features) da variável alvo (target). Em seguida, faço a divisão dos dados em conjuntos de treino e teste

In [3]:
# Separar as features e o target
X = df.drop('IMDB_Rating', axis=1)
y = df['IMDB_Rating']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Pré-processamento e Tratamento dos Dados
Com todo o conhecimento que já possuo dos dados após ter feito toda a análise exploratória, faço os passos de pré-processamento e tratamento dos dados. Em seguida, defino transformadores específicos para cada tipo de característica para garantir uma predição mais robusta dos dados.

Gostaria de falar um pouco sobre a técnica para preencher valores ausentes um pouco mais avançada que estou utilizando, que é o KNNImputer:

O KNNImputer é uma técnica avançada de imputação de valores faltantes que utiliza os valores dos vizinhos mais próximos (k-nearest neighbors) para imputar os valores ausentes. Aqui estão algumas vantagens do KNNImputer em comparação com métodos mais simples, como a imputação com a média ou mediana:

Considera a Similaridade dos Dados:

- O KNNImputer leva em conta a similaridade entre os dados ao imputar valores faltantes. Isso significa que a imputação é baseada nos valores de pontos de dados semelhantes (os vizinhos mais próximos), o que pode resultar em imputações mais precisas e contextualmente apropriadas.

In [4]:
# Identificar características numéricas e categóricas no conjunto de dados X
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Definir transformadores para características numéricas e categóricas
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),  # Preencher valores ausentes com KNN Imputer
    ('normalização', StandardScaler())])          # Normalizar os valores numéricos (padronização)

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Preencher valores ausentes com o valor mais frequente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])   # Codificar variáveis categóricas usando one-hot encoding

# Combinar transformadores em um pré-processador de colunas
preprocessor = ColumnTransformer(
    transformers=[
        ('numérica', numeric_transformer, numeric_features),     # Aplicar transformação numérica às características numéricas
        ('categórica', categorical_transformer, categorical_features)])  # Aplicar transformação categórica às características categóricas

### Definição e Otimização dos Modelos de Regressão
Defino os modelos base de regressão e as distribuições de parâmetros para otimização de hiperparâmetros. Utilizo o RandomizedSearchCV para encontrar os melhores estimadores.

In [5]:
# Definição dos modelos base de regressão
models = {
    'LR': LinearRegression(),
    'DT': DecisionTreeRegressor(),
    'RF': RandomForestRegressor(),
    'GB': GradientBoostingRegressor(),
    'MLP': MLPRegressor(),
    'SVR': SVR()
}

# Definição das distribuições de parâmetros para otimização de hiperparâmetros
param_distributions = {
    'LR': {}, # Não consegui encontrar parâmetros da regressão linear para aplicar otimizações
    'DT': {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 10, 20]},
    'RF': {'n_estimators': [50, 100, 200], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10]},
    'GB': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 4, 5]},
    'MLP': {'hidden_layer_sizes': [(50,50), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']},
    'SVR': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
}

# Otimização de hiperparâmetros e treinamento dos melhores estimadores
best_estimators = {}
for name, model in models.items():
    # Configuração do RandomizedSearchCV para o modelo atual
    search = RandomizedSearchCV(model, param_distributions[name], n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=42)

    # Construção do pipeline com pré-processamento e modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), (name, search)])

    # Treinamento do pipeline
    pipeline.fit(X_train, y_train)

    # Armazenamento do melhor estimador encontrado pelo RandomizedSearchCV
    best_estimators[name] = pipeline.named_steps[name].best_estimator_

### Criação do Ensemble (Comitê de Modelos) Voting Regressor

Com os melhores estimadores encontrados, crio um ensemble de regressão usando o **Voting Regressor**.

In [6]:
# Criar o Voting Regressor com os melhores estimadores
voting_regressor = VotingRegressor(estimators=[
    ('LR', best_estimators['LR']),
    ('DT', best_estimators['DT']),
    ('RF', best_estimators['RF']),
    ('GB', best_estimators['GB']),
    ('MLP', best_estimators['MLP']),
    ('SVR', best_estimators['SVR'])
])

### Treinamento do Modelo Final

Construo o pipeline final, que inclui o pré-processamento e o ensemble voting regressor, e realizo o treinamento do modelo.

In [7]:
# Pipeline final
final_pipeline = Pipeline(steps=[('Pré-Processamento', preprocessor), ('Ensemble Voting', voting_regressor)])
# Fazendo o treinamento do modelo desenvolvido
final_pipeline.fit(X_train, y_train)

Pipeline(steps=[('Pré-Processamento',
                 ColumnTransformer(transformers=[('numérica',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('normalização',
                                                                   StandardScaler())]),
                                                  Index(['Meta_score', 'No_of_Votes'], dtype='object')),
                                                 ('categórica',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(...
                 VotingRegressor(estimators=[('LR', LinearRegression()),
                                             ('DT',
                                              DecisionTreeRegressor(max_depth=10,
                                                                    min_samples_split=20)),
                                             ('RF',
                                              RandomForestRegressor(max_depth=30,
                                                                    max_features='sqrt',
                                                                    min_samples_split=5,
                                                                    n_estimators=200)),
                                             ('GB',
                                              GradientBoostingRegressor(max_depth=5)),
                                             ('MLP',
                                              MLPRegressor(activation='tanh',
                                                           hidden_layer_sizes=(50,
                                                                               50),
                                                           solver='sgd')),
                                             ('SVR',
                                              SVR(C=0.1, kernel='linear'))]))])

### Avaliação do Modelo
Após isso, realizo as previsões no conjunto de teste e calculo as métricas de avaliação para entender a performance do modelo.

In [8]:
# Realização da previsão com o modelo final e avaliação dos resultados

y_pred = final_pipeline.predict(X_test)  # Realiza previsões usando o modelo finalizado no conjunto de teste que foi separado e que não sofreu interferências

# Cálculo das métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)  # Calcula o erro médio absoluto
mse = mean_squared_error(y_test, y_pred)   # Calcula o erro quadrático médio
rmse = np.sqrt(mse)                        # Calcula a raiz do erro quadrático médio (RMSE)
r2 = r2_score(y_test, y_pred)              # Calcula o coeficiente de determinação R^2

# Apresentado os resultados
print(f'MAE: {round(mae,2)}')
print(f'MSE: {round(mse,2)}')
print(f'RMSE: {round(rmse,2)}')
print(f'R^2: {round(r2,2)}')

MAE: 0.18
MSE: 0.05
RMSE: 0.22
R^2: 0.27


### Análise das Métricas de Avaliação

#### MAE (Erro Médio Absoluto)

- Valor: 0.18
- Interpretação: Em média, as previsões do modelo estão erradas por cerca de 0.18 pontos na escala de classificação do IMDb. Isso significa que, em média, a diferença entre os valores previstos e os valores reais é de aproximadamente 0.18.

#### MSE (Erro Quadrático Médio)

- Valor: 0.05
- Interpretação: O MSE penaliza erros maiores de forma mais severa, já que os erros são elevados ao quadrado antes de serem somados. Um MSE de 0.05 sugere que os erros do modelo, em média, são baixos, mas é mais difícil de interpretar diretamente em comparação com o MAE.

#### RMSE (Raiz do Erro Quadrático Médio)

- Valor: 0.22
- Interpretação: A RMSE é a raiz quadrada do MSE e é mais interpretável, pois está na mesma escala que as variáveis alvo (IMDb ratings). Um RMSE de 0.22 indica que, em média, as previsões do modelo estão a cerca de 0.22 pontos de distância dos valores reais do IMDb.

#### R² (Coeficiente de Determinação)

- Valor: 0.27
- Interpretação: O valor de R² indica a proporção da variância nos dados alvo que é explicada pelo modelo. Um R² de 0.27 significa que aproximadamente 27% da variabilidade nos ratings do IMDb pode ser explicada pelas features incluídas no modelo.

#### Conclusão
Sendo assim, o modelo que construí possui poder preditivo, mas possui espaço para melhoria, como normalmente todo modelo preditivo possui.

### Salvando o modelo desenvolvido em formato .pkl

In [9]:
# Salvando o modelo
joblib.dump(final_pipeline, 'avaliacao_imdb_model.pkl')

['avaliacao_imdb_model.pkl']

### Predição da avaliação IMDb do filme 'The Shawshank Redemption'

In [10]:
# Carregando o modelo salvo para fazer a predição de avaliação do filme
modelo_predicao = joblib.load('avaliacao_imdb_model.pkl')

In [12]:
# Características do novo filme
novo_filme = {
    'Series_Title': 'The Shawshank Redemption',
    'Released_Year': '1994',
    'Certificate': 'A',
    'Runtime': '142 min',
    'Genre': 'Drama',
    'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
    'Meta_score': 80.0,
    'Director': 'Frank Darabont',
    'Star1': 'Tim Robbins',
    'Star2': 'Morgan Freeman',
    'Star3': 'Bob Gunton',
    'Star4': 'William Sadler',
    'No_of_Votes': 2343110,
    'Gross': '28,341,469'
}

# Criar um DataFrame com as características do novo filme
novo_filme_df = pd.DataFrame([novo_filme])

# Fazer a previsão
predicted_imdb_score = modelo_predicao.predict(novo_filme_df)

print(f'A nota de avaliação IMDb do filme The Shawshank Redemption, foi: {round(predicted_imdb_score[0],2)}!')

A nota de avaliação IMDb do filme The Shawshank Redemption, foi: 8.73!


## Resoluções e Análises Conclusivas

### Variáveis Utilizadas e suas Transformações:
- Variáveis Numéricas: Foram normalizadas para garantir que todas as variáveis estejam na mesma escala, o que é especialmente importante para modelos de machine learning que utilizam métricas de distância (e.g., SVM).

- Variáveis Categóricas: Foram codificadas usando one-hot encoding para converter categorias em uma forma que os algoritmos de machine learning podem utilizar.

### Tipo de Problema:
Estou resolvendo um problema de **Regressão**, onde o objetivo é prever a nota do IMDb, que é uma variável contínua.

### Modelo que Melhor se Aproxima dos Dados:
Apliquei um **Ensemble Voting Regressor** composto pelos melhores estimadores de vários modelos utilizando a técnica de otimização do random search.

##### Acho importante trazer também uma **introdução** sobre o modelo que escolhi e apliquei para um melhor entendimento:

O Voting Regressor é uma técnica de ensemble learning usada para melhorar a precisão e a robustez dos modelos preditivos. Em essência, o Voting Regressor combina previsões de múltiplos modelos de regressão individuais (também chamados de "estimadores base") para fazer uma previsão final. O conceito central por trás do Voting Regressor é que diferentes modelos podem capturar diferentes padrões nos dados e, ao combinar suas previsões, podemos obter uma predição mais precisa e estável do que qualquer modelo individual.

**Vantagens do Voting Regressor:**
- Robustez: Ao combinar múltiplos modelos, o Voting Regressor tende a ser mais robusto contra outliers e variações nos dados.
- Redução de Overfitting: Modelos individuais podem superajustar os dados de treinamento, mas a combinação de modelos pode ajudar a suavizar as previsões e reduzir o risco de overfitting.
- Melhora na Precisão: A agregação das previsões de diferentes modelos frequentemente resulta em uma melhora na precisão geral das previsões.
- Flexibilidade: Podemos combinar qualquer conjunto de modelos de regressão, incluindo modelos lineares, árvores de decisão, redes neurais, etc.

**Além disso, trago uma pequena introdução também sobre a técnica de otimização de hiperparâmetros random search:**

- O Randomized Search é uma técnica de otimização de hiperparâmetros usada para melhorar a performance de modelos de aprendizado de máquina. Hiperparâmetros são parâmetros cujos valores são definidos antes do treinamento do modelo e podem impactar significativamente sua performance. O Randomized Search ajuda a encontrar a combinação ideal desses hiperparâmetros de forma eficiente.

### Medidas de Performance Escolhidas e Por Quê:

- MAE (Erro Médio Absoluto): Fácil de interpretar, mostra o erro médio das previsões em unidades da variável alvo.
- MSE (Erro Quadrático Médio): Penaliza erros maiores mais severamente, útil para identificar grandes discrepâncias.
- RMSE (Raiz do Erro Quadrático Médio): Fácil de interpretar (na mesma escala que a variável alvo) e útil para identificar a magnitude dos erros.
- R² (Coeficiente de Determinação): Indica a proporção da variabilidade dos dados que é explicada pelo modelo, fornecendo uma medida de quão bem o modelo está capturando a variância nos dados.

Logo, essas métricas foram escolhidas para fornecer uma visão abrangente da performance do modelo, considerando tanto a magnitude dos erros quanto a capacidade do modelo de explicar a variância nos dados.